In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

In [2]:
#constants
SEQUENCE_LEN = 100

TODO:

- die dictionaries sollen funktional, unabhaängig vond er klasse sein, also werden die songs als solche ebreits auserhalb genereirt, dann einfach songs drausen generieren, dict anlegen, dann klasse schreiben die songs bekommt

- https://github.com/LeanManager/NLP-PyTorch/blob/master/Character-Level%20LSTM%20with%20PyTorch.ipynb
- https://www.analyticsvidhya.com/blog/2020/08/build-a-natural-language-generation-nlg-system-using-pytorch/

# Load and prepare Data

Lets begin by importing the data.

In [3]:
file = 'eurobeat_lyrics.csv'

songs  = np.array(open(file,'r').read().split(';'))
songs = songs[:-1]

In [4]:
#function to call in dataset class
def getSongs(filepath):
    songs  = np.array(open(file,'r').read().split(';'))
    return songs[:-1]

The songs have different lengths. We want to split the songs in sequences of characters, all the same length. We also want the sequences to overlap.

In [5]:
#get a whole song and return an array (n, seq_len) of n seqeunces of length seq_len, which are the characters of the song
#each sequence has teh same length
def song2sequences(song, seq_len):
    song_len = len(song)
    n_seq = 2*(song_len//seq_len)
    seq = np.empty((n_seq, seq_len), dtype=str)
    
    song = np.array([ch for ch in song])
    for i in range(n_seq-1):
        seq[i] = song[int(i*(seq_len/2)) : int(seq_len + i*seq_len/2)]
    seq[n_seq-1] = song[-(seq_len):]
    
    return seq

#lets create all the sequences for all our songs and concatenate them in one array
#so proud i wrote this line myself
def songs2sequences(songs, seq_len=SEQUENCE_LEN):
    return np.concatenate(([song2sequences(song, seq_len) for song in songs]))

In [6]:
#sequences = songs2sequences(songs,SEQUENCE_LEN)

## Tokenization

Create a dictionary to link letters to numbers.

A `set` removes all dublicate elements. We want a dictionary we can use for all songs therefore we must create a set from all songs.

In [35]:
#set of all chars
chars = set(' '.join(songs))

In [37]:
#dictionaries, linking a integer to a symbol
int2char_dict = dict(enumerate(chars))
char2int_dict = { int2char[k]:k for k in int2char}
#print(int2char_dict[6])
#print(char2int_dict['E'])

Now turn the sequences into integers. We then want to use `OneHot` of pytorch to encode the sequences as Propability Mass Vectors.

The resulting tensor `sequences_OH` does now have the shape (seq_n, seq_len, len(chars)) as each character is represented by a len(chars)-long base vector.

In [9]:
#turn the sequences into integers by dictionary
#sequences_int = torch.tensor([[char2int_dict[char] for char in seq] for seq in sequences])
#print(sequences_int.dtype)

#one-hot encoding
#sequences_OH = F.one_hot(sequences_int)
#print(sequences_OH[0,0])
#print(sequences_int[0,0])
#print(sequences[0,0])

In [10]:
#sequences_int[0]
#print(''.join([int2char[ch] for ch in sequences_int[0]]))

def OneHotSequence2String(seq):
    seq = seq.numpy()
    seq = np.argmax(seq, axis=1)
    return ''.join([int2char_dict[ch] for ch in seq])

Now define a custom Dataset-Class and DataLoader.

In [43]:
class LyricsDataset(Dataset):
    def __init__(self, songs, seq_len, dict_len, int2char, char2int): #constructor
        
        self.seq_len = seq_len
        self.dict_len = dict_len
        self.int2char = int2char
        self.char2int = char2int
        
        
        self.songs = songs
        self.sequences_raw = songs2sequences(self.songs, self.seq_len)
        #turn the sequences into integers by dictionary
        self.sequences_int = torch.tensor([[self.char2int[char] for char in seq] for seq in self.sequences_raw])
        #one-hot encoding
        self.sequences = F.one_hot(self.sequences_int).to(float)
        
        #target: forward(sequences[i]) predicts sequences_target[i]=sequences[i+1]
        #take the raw sequence and advance one element
        self.sequences_target_raw = np.roll(self.sequences_raw,-1)
        self.sequencess_target_int = torch.tensor([[self.char2int[char] for char in seq] for seq in self.sequences_target_raw])
        self.sequences_target = F.one_hot(self.sequencess_target_int).to(float)


    def __len__(self): 
        return self.sequences.shape[0] #return the number of sequences
    
    def __getitem__(self, idx): 
        return self.sequences[idx], self.sequences_target[idx] #return the sequences and targets

In [50]:
dataset = LyricsDataset(songs,SEQUENCE_LEN, len(chars), int2char_dict, char2int_dict)
dataloader = DataLoader(dataset, batch_size=5)

In [63]:
for seq, tar in dataloader:
    print(tar.shape)
    print(OneHotSequence2String(seq[0]))
    print('----------')
    print(OneHotSequence2String(tar[0]))
    break

torch.Size([5, 100, 67])
Lost in my reaction

Dancing like a Parapara star in Japan

My name is like a thunder

Baby, you can
----------
ost in my reaction

Dancing like a Parapara star in Japan

My name is like a thunder

Baby, you cani


# Define a model

In [49]:
class LSTMNetwork(torch.nn.Module):
    #seq_length: length of seqeunce: 100
    # char_dim: length of hot ones base vectors/length of dictionary
    #hidden layer dim
    def __init__(self, sequence_dim, char_dim, hidden_dim, n_layers, dropout_prob):
        super(LSTMTagger, self).__init__()
        self.sequence_dim = sequence_dim
        self.char_dim = char_dim
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.dropout_prob = dropout_prob

        #lstm
        self.lstm = torch.nn.LSTM(input_size=self.char_dim, hidden_size=self.hidden_dim,
                                 num_layers=self.n_layers, batch_first=True,
                                 dropout=self.dropout_prob)
        

        # The linear layer that maps from hidden state space to char-propability space
        self.lin = torch.nn.Linear(self.hidden_dim, self.char_dim)
        
        #init the hidden layer, weights and bias of lstm and lin

    def forward(self, x_n, hc_prev):
        
        y_n, (h_n, c_n) = self.lstm(x_n, hc_prev)
        
        #stack up the 2 lstm cells, send output of fisrt into second 
        y_n = y_n.view(y_n.size()[0]*y_n.size()[1], self.hidden_dim)
        
        ## Put y_n through the output
        y_n = self.fc(y_n)
        
        return y_n, (h_n, c_n)
    
    
    def init_hidden(self, batch_size):
        # Create two new tensors with sizes (n_layers, n_seqs, hidden_dim)
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        return (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_(),
                weight.new(self.n_layers, batch_size, self.hidden_dim).zero_())
        

# Training process

1. Beginn by defining:
- `opt` The optimization algorithm (Gradient decent).
- `loss_function` Evaluate the accuracy of the current state with the Cross Entropy Function.
2. Next, set the model into training mode, this is only relevant for the Dropout Layer.
3. We will loop over the whole data several times with the `epochs` loop.
4. For every epoch the hidden layer needs to be reset.
5. Now we loop over the batches of our dataset. We have a `target` batch with which we will later compare the processed `input`.
6. Reset the Gradient and reintroduce the hidden state, this prevents the backwards propagation from going beyond the current state.
7. Now finally, calculate the predictions `output`. This also returns the hidden state `h`.
8. Perform the backwards propagation:
- Calculate the loss.
- Call `backward()`.
- `clip_grad_norm_` prevents the parameters from exploding.
- update the parameters.
9. print the training status.

In [57]:
def training(net, data, sequence_dim, dict_len, batch_size,  epochs=10, lr=0.01):
    # optimizer
    opt = torch.optim.SGD(net.parameters(), lr=lr)
    # loss
    loss_function = torch.nn.CrossEntropyLoss()

    #need to set the mode
    net.train()
    
    counter = 0
    
    for e in range(epochs):

        #initialize hidden state
        h = net.init_hidden(batch_size)
        
        for inputs, targets in data:
            counter+= 1
            
            #reset the gradients
            net.zero_grad()
            #reentroduce the hidden state
            h = tuple([each.data for each in h])
       
            #get the output from the model
            outputs, h = net.forward(inputs, h)
            
            #TODO is softmax included in cross entropy? otherwise here
            
            #calculate the loss and perform backprop
            loss = loss_function(outputs, targets)
            #back-propagate error
            loss.backward()
            #helps prevent the exploding gradient problem in RNNs / LSTMs.
            torch.nn.utils.clip_grad_norm_(net.parameters(), clip)
            # update weigths
            opt.step()            
            
            if counter % print_every == 0:
            
              print("Epoch: {}/{}...".format(e+1, epochs),
                    "Step: {}...".format(counter))